In [1]:
import sys
sys.path.insert(1, './env')
sys.path.insert(1, './agents')

import gym
import pylab
import random
import strategy_imitation, sarsa, ddqn, random_agent, a2c, model_based
import aa_gun
import numpy as np
from collections import deque
import keras
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


In [ ]:
#массовый тест моделей

In [ ]:
#Проверь на зенитке, на cartpole и на mountain car

score_dict_full={'random':[],'ddqn':[],'a2c':[],'sarsa':[],'mb':[]}
border_med = 100
EPISODES=100
score_dict_med={'random':[],'ddqn':[],'a2c':[],'sarsa':[],'mb':[]}#c border_med по... такты. Надо, чтобы проверить быстроту обучения

agent_list=[random_agent.randomAgent,a2c.A2CAgent,ddqn.DoubleDQNAgent,sarsa.SarsaAgent,model_based.ModelBasedAgent]
for ag_num in range(4,5):
    if ag_num==0:
        name='random'
    if ag_num==1:
        name='a2c'
    if ag_num==2:
        name='ddqn' 
    if ag_num==3:
        name='sarsa'
    if ag_num==4:
        name='mb'
    
    for estimation in range(3):
        print('_____',name,estimation,pd.Timestamp.now())
        #здесь весь код от инициализации модели до выдачи scores. Но без рендера.
        # In case of CartPole-v1, maximum length of episode is 500
        
        env = aa_gun.AA_gun_simple0_env()
        #env = gym.make('Seaquest-ramNoFrameskip-v0')
        #env=CartPoleEnv9()
        # get size of state and action from environment
        state_size = env.observation_space.shape[0]
        action_size = env.action_space.n

        #agent = DoubleDQNAgent(state_size, action_size)
        agent = agent_list[ag_num](state_size, action_size)
        agent.train_start=1000
        #agent.train_start=7000
        agent.epsilon_decay=0.9999
        agent.render=False

        scores, episodes = [], []
        reward_lst = []
        s_list=[]
        a_list=[]

        for e in range(EPISODES):
            done = False
            score = 0
            state = env.reset()
            state = np.reshape(state, [1, state_size])

            while not done:
                #if (e in range(2,7)) or (e in range(20,25)) or (e in range(100,103)) or (e in range(200,202)) or (e in range(300,306)) or (e in range(400,406)) or (e in range(500,506)) or (e in range(600,604)):
                #    if agent.render:
                #        env.render()

                # get action for the current state and go one step in environment
                action = agent.get_action(state)
                next_state, reward, done, info = env.step(action)
                next_state = np.reshape(next_state, [1, state_size])
                # if an action make the episode end, then gives penalty of -100

                
                # save the sample <s, a, r, s'> to the replay memory
                agent.append_sample(state, action, reward, next_state, done)
                #if next_state[0,11]!=reward:
                #    print('state[13]!=reward',state[0,11],reward)
                #
                s_list.append(state)
                a_list.append(action)
                reward_lst.append(reward)
                #

                # every time step do the training
                agent.train_model()
                score += reward
                state = next_state

                if done:
                    # every episode update the target model to be same with model
                    agent.update_target_model()

                    # every episode, plot the play time
                    scores.append(score)
                    episodes.append(e)
                    pylab.plot(episodes, scores, 'b')
                    #pylab.savefig("./save_graph/aa_gun_dqn.png")
                    print("episode:", e, "  score:", score,np.mean(scores), "  memory length:",
                          len(agent.s), "  epsilon:", agent.epsilon)



            # save the model
            #if e % 50 == 0:
            #    agent.model.save_weights("./save_model/aa_gun_dqn.h5")



        #и первые 3000 тактов - это рандом
        #Ходов так 50
        score_dict_full[name].append(np.mean(scores))
        score_dict_med[name].append(np.mean(scores[border_med:]))
        import pickle
        f=open('score_dict.pkl','wb')
        pickle.dump([score_dict_full,score_dict_med],f)
        plt.show()

_____ mb 0 2020-01-30 18:00:07.028938
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 260)               5200      
_________________________________________________________________
batch_normalization_1 (Batch (None, 260)               1040      
_________________________________________________________________
dropout_1 (Dropout)          (None, 260)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 260)               67860     
_________________________________________________________________
batch_normalization_2 (Batch (None, 260)               1040      
_________________________________________________________________
dropout_2 (Dropout)          (None, 260)               0         
_________________________________________________________________
dense_3 (Dense) 

Epoch 50/60
1000/1000 [==============================] - 0s 19us/step - loss: 7715.9663
Epoch 51/60
1000/1000 [==============================] - 0s 19us/step - loss: 7719.3296
Epoch 52/60
1000/1000 [==============================] - 0s 32us/step - loss: 7702.8306
Epoch 53/60
1000/1000 [==============================] - 0s 28us/step - loss: 7684.5669
Epoch 54/60
1000/1000 [==============================] - 0s 20us/step - loss: 7677.8013
Epoch 55/60
1000/1000 [==============================] - 0s 19us/step - loss: 7661.6323
Epoch 56/60
1000/1000 [==============================] - 0s 19us/step - loss: 7640.5933
Epoch 57/60
1000/1000 [==============================] - 0s 20us/step - loss: 7633.1650
Epoch 58/60
1000/1000 [==============================] - 0s 19us/step - loss: 7618.4863
Epoch 59/60
1000/1000 [==============================] - 0s 20us/step - loss: 7607.1919
Epoch 60/60
1000/1000 [==============================] - 0s 19us/step - loss: 7587.4375
Epoch 1/60
1000/1000 [==========

Epoch 24/60
1000/1000 [==============================] - 0s 20us/step - loss: 6360.0459
Epoch 25/60
1000/1000 [==============================] - 0s 19us/step - loss: 6336.6821
Epoch 26/60
1000/1000 [==============================] - 0s 20us/step - loss: 6335.4609
Epoch 27/60
1000/1000 [==============================] - 0s 19us/step - loss: 6318.6934
Epoch 28/60
1000/1000 [==============================] - 0s 20us/step - loss: 6292.0757
Epoch 29/60
1000/1000 [==============================] - 0s 20us/step - loss: 6293.5054
Epoch 30/60
1000/1000 [==============================] - 0s 19us/step - loss: 6278.7783
Epoch 31/60
1000/1000 [==============================] - 0s 18us/step - loss: 6273.2090
Epoch 32/60
1000/1000 [==============================] - 0s 18us/step - loss: 6228.6050
Epoch 33/60
1000/1000 [==============================] - 0s 19us/step - loss: 6218.7397
Epoch 34/60
1000/1000 [==============================] - 0s 21us/step - loss: 6205.8545
Epoch 35/60
1000/1000 [=========

Epoch 58/60
1000/1000 [==============================] - 0s 19us/step - loss: 4926.0093
Epoch 59/60
1000/1000 [==============================] - 0s 17us/step - loss: 4928.1631
Epoch 60/60
1000/1000 [==============================] - 0s 19us/step - loss: 4907.3203
Epoch 1/60
1000/1000 [==============================] - 0s 19us/step - loss: 4903.6211
Epoch 2/60
1000/1000 [==============================] - 0s 21us/step - loss: 4856.6748
Epoch 3/60
1000/1000 [==============================] - 0s 22us/step - loss: 4855.6519
Epoch 4/60
1000/1000 [==============================] - 0s 21us/step - loss: 4828.8340
Epoch 5/60
1000/1000 [==============================] - 0s 21us/step - loss: 4832.9546
Epoch 6/60
1000/1000 [==============================] - 0s 21us/step - loss: 4814.4395
Epoch 7/60
1000/1000 [==============================] - 0s 22us/step - loss: 4806.3101
Epoch 8/60
1000/1000 [==============================] - 0s 19us/step - loss: 4787.9106
Epoch 9/60
1000/1000 [==================

Epoch 32/60
1000/1000 [==============================] - 0s 20us/step - loss: 3485.6733
Epoch 33/60
1000/1000 [==============================] - 0s 20us/step - loss: 3468.4216
Epoch 34/60
1000/1000 [==============================] - 0s 21us/step - loss: 3433.4495
Epoch 35/60
1000/1000 [==============================] - 0s 19us/step - loss: 3426.7861
Epoch 36/60
1000/1000 [==============================] - 0s 19us/step - loss: 3405.8889
Epoch 37/60
1000/1000 [==============================] - 0s 18us/step - loss: 3402.8311
Epoch 38/60
1000/1000 [==============================] - 0s 19us/step - loss: 3390.9873
Epoch 39/60
1000/1000 [==============================] - 0s 19us/step - loss: 3400.9065
Epoch 40/60
1000/1000 [==============================] - 0s 20us/step - loss: 3359.0117
Epoch 41/60
1000/1000 [==============================] - 0s 18us/step - loss: 3373.7024
Epoch 42/60
1000/1000 [==============================] - 0s 21us/step - loss: 3347.4890
Epoch 43/60
1000/1000 [=========

...

500/500 [==============================] - 0s 30us/step - loss: 637.6326
Epoch 1/2
500/500 [==============================] - 0s 26us/step - loss: 635.5195
Epoch 2/2
500/500 [==============================] - 0s 27us/step - loss: 605.7468
self-test 3.8809451739750607
Epoch 1/2
500/500 [==============================] - 0s 26us/step - loss: 729.7952
Epoch 2/2
500/500 [==============================] - 0s 30us/step - loss: 710.5094
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 731.9237
Epoch 2/2
500/500 [==============================] - 0s 29us/step - loss: 694.1666
Epoch 1/2
500/500 [==============================] - 0s 29us/step - loss: 703.9182
Epoch 2/2
500/500 [==============================] - 0s 27us/step - loss: 699.5141
Epoch 1/2
500/500 [==============================] - 0s 28us/step - loss: 696.5037
Epoch 2/2
500/500 [==============================] - 0s 27us/step - loss: 703.6016
Epoch 1/2
500/500 [==============================] - 0s 28us/step - 

500/500 [==============================] - 0s 27us/step - loss: 551.8698
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 532.4229
Epoch 1/2
500/500 [==============================] - 0s 30us/step - loss: 554.3981
Epoch 2/2
500/500 [==============================] - 0s 27us/step - loss: 528.1271
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 534.2161
Epoch 2/2
500/500 [==============================] - 0s 27us/step - loss: 529.0362
self-test 3.034263368743097
Epoch 1/2
500/500 [==============================] - 0s 30us/step - loss: 687.8681
Epoch 2/2
500/500 [==============================] - 0s 26us/step - loss: 689.0182
Epoch 1/2
500/500 [==============================] - 0s 26us/step - loss: 652.6197
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 672.2053
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 658.5350
Epoch 2/2
500/500 [==============================] - 0s 29us/step - l

500/500 [==============================] - 0s 28us/step - loss: 630.8756
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 602.8145
Epoch 2/2
500/500 [==============================] - 0s 32us/step - loss: 644.9983
Epoch 1/2
500/500 [==============================] - 0s 29us/step - loss: 652.9980
Epoch 2/2
500/500 [==============================] - 0s 32us/step - loss: 664.6333
Epoch 1/2
500/500 [==============================] - 0s 49us/step - loss: 601.3506
Epoch 2/2
500/500 [==============================] - 0s 48us/step - loss: 635.1276
Epoch 1/2
500/500 [==============================] - 0s 53us/step - loss: 634.2249
Epoch 2/2
500/500 [==============================] - 0s 87us/step - loss: 606.6696
self-test 3.6239138154285895
Epoch 1/2
500/500 [==============================] - 0s 41us/step - loss: 774.0579
Epoch 2/2
500/500 [==============================] - 0s 27us/step - loss: 791.8397
Epoch 1/2
500/500 [==============================] - 0s 30us/step - 

500/500 [==============================] - 0s 28us/step - loss: 627.5130
Epoch 2/2
500/500 [==============================] - 0s 30us/step - loss: 652.8811
Epoch 1/2
500/500 [==============================] - 0s 29us/step - loss: 631.8812
Epoch 2/2
500/500 [==============================] - 0s 27us/step - loss: 645.0128
Epoch 1/2
500/500 [==============================] - 0s 29us/step - loss: 606.6215
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 614.9597
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 638.2737
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 613.4538
Epoch 1/2
500/500 [==============================] - 0s 29us/step - loss: 586.3112
Epoch 2/2
500/500 [==============================] - 0s 27us/step - loss: 610.3235
Epoch 1/2
500/500 [==============================] - 0s 78us/step - loss: 618.4650
Epoch 2/2
500/500 [==============================] - 0s 52us/step - loss: 606.3120
self-test 3.57

500/500 [==============================] - 0s 29us/step - loss: 704.6915
Epoch 1/2
500/500 [==============================] - 0s 30us/step - loss: 687.1182
Epoch 2/2
500/500 [==============================] - 0s 30us/step - loss: 718.1066
Epoch 1/2
500/500 [==============================] - 0s 29us/step - loss: 740.8225
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 706.6046
Epoch 1/2
500/500 [==============================] - 0s 28us/step - loss: 682.7465
Epoch 2/2
500/500 [==============================] - 0s 29us/step - loss: 742.4106
Epoch 1/2
500/500 [==============================] - 0s 31us/step - loss: 688.0616
Epoch 2/2
500/500 [==============================] - 0s 31us/step - loss: 730.4697
Epoch 1/2
500/500 [==============================] - 0s 32us/step - loss: 669.0690
Epoch 2/2
500/500 [==============================] - 0s 30us/step - loss: 660.7266
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 671.5319
Epoch 2/2
500/

In [ ]:
score_dict_full